In [10]:
%ls

__pycache__/         kelp_prediction.csv  price_round.ipynb
factor_pipeline.py   main.py              trade_round.ipynb


In [13]:
%ls ../../../data/round-1-island-data-bottle/

prices_round_1_day_-1.csv  trades_round_1_day_-1.csv
prices_round_1_day_-2.csv  trades_round_1_day_-2.csv
prices_round_1_day_0.csv   trades_round_1_day_0.csv


In [18]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('../../../data/round-1-island-data-bottle/trades_round_1_day_-2.csv', delimiter=';')
df2 = pd.read_csv('../../../data/round-1-island-data-bottle/trades_round_1_day_-1.csv', delimiter = ';')
df3 = pd.read_csv('../../../data/round-1-island-data-bottle/trades_round_1_day_0.csv', delimiter = ';')

df = pd.concat([df1, df2, df3])

In [19]:
print(df)

      timestamp  buyer  seller            symbol   currency   price  quantity
0             0    NaN     NaN              KELP  SEASHELLS  2002.0         1
1             0    NaN     NaN  RAINFOREST_RESIN  SEASHELLS  9996.0         1
2             0    NaN     NaN  RAINFOREST_RESIN  SEASHELLS  9996.0         1
3             0    NaN     NaN         SQUID_INK  SEASHELLS  2002.0         1
4           100    NaN     NaN              KELP  SEASHELLS  2002.0         6
...         ...    ...     ...               ...        ...     ...       ...
8177     999500    NaN     NaN              KELP  SEASHELLS  2032.0         2
8178     999500    NaN     NaN  RAINFOREST_RESIN  SEASHELLS  9995.0         2
8179     999500    NaN     NaN         SQUID_INK  SEASHELLS  1830.0         2
8180     999600    NaN     NaN              KELP  SEASHELLS  2032.0         1
8181     999600    NaN     NaN         SQUID_INK  SEASHELLS  1832.0         1

[24886 rows x 7 columns]
